# ColorCamp

In [18]:
import colorcamp as cc
from tempfile import TemporaryDirectory

## Working with colors and color objects
All colors and color objects can be tagged with an additional `name`, `description`, and unstructured `metadata`.

### Colors
Color camp allows users to represent colors in common color spaces: Hex, RGB, HSL. Colors are extensions of the common native type, e.g. Hex == `str`, RGB == `tuple`.


In [19]:
mustard = cc.Hex('#FFAA15', name = 'mustard')
sky = cc.Hex('#15AAFF', name = 'sky')
lime = cc.Hex('#15FFAA', name = 'lime')
pink = cc.Hex('#FF15AA', name = 'pink')

# default representation
print(mustard)
# accessing string methods
print(mustard.lower())
print(f"is mustard a string: {isinstance(mustard, str)}")
# notebook html representation
mustard


#FFAA15
#ffaa15
is mustard a string: True


'#FFAA15'

Colors can be converted between colorspaces.

In [20]:
mustard_hsl = mustard.to_hsl()
mustard_rgb = mustard.to_rgb()
display(mustard_hsl, mustard_rgb)

(38.205128, 1.0, 0.541176)

(255, 170, 21)

Alpha can be adjusted on colors

In [21]:
# Note this returns a new color object
mustard_rgb.change_alpha(0.7)

(255, 170, 21, 0.7)

Colors have many useful attributes and methods

In [22]:
new_pink = cc.RGB(
    (255,21,170), 
    name = 'super_pink',
    description="An energizing pink",
    metadata={"sentiment": "excited"},
)

display(new_pink)

print(
    f'{new_pink.name} red value: {new_pink.red}\n',
    f'{new_pink.name} hex string: {new_pink.hex}\n',
    f'{new_pink.name} css: {new_pink.css()}\n',
    f'{new_pink.name} alpha: {new_pink.alpha}\n',
    f'{new_pink.name} alpha: {new_pink.info()}\n',

)

(255, 21, 170)

super_pink red value: 255
 super_pink hex string: #FF15AA
 super_pink css: rgb(255, 21, 170)
 super_pink alpha: None
 super_pink alpha: {'name': 'super_pink', 'description': 'An energizing pink', 'metadata': {'sentiment': 'excited'}}



Colors can be added together to blend them together. The resulting color is casted as the left most color. They can also be tested in equalities

In [23]:
color_1 = new_pink + mustard
color_2 = mustard + new_pink

display(color_1, color_2)

new_pink == pink

(255, 96, 96)

'#FF6060'

True

### Color Objects
Other color objects are different ways of representing groups of colors and include: Palettes, Scales, and Maps

### Palettes
Color palettes are ordered tuples of colors, useful for categorical values, brand colors, mood boards, etc.

In [24]:
brand_colors = cc.Palette(
    colors = [
        mustard,  
        lime, 
        sky,
        pink,
    ],
    name = 'bright_and_sunny'
)

# standard representation
print(brand_colors)
# html representation
brand_colors

Palette('#FFAA15', '#15FFAA', '#15AAFF', '#FF15AA')


Palette('#FFAA15', '#15FFAA', '#15AAFF', '#FF15AA')

### Scales
Scales are similar to palettes, but they imply continuous data, gradients, and divergent data. Scales will infer linear interpolation between two colors. The relative distances between them can be adjusted with `stops`

In [25]:
# Scales can be created from palettes
brand_scale = cc.Scale(brand_colors, name = 'BrandScale')
uneven_brand_scale = cc.Scale(brand_colors, stops=[0, 0.75, 0.9, 1])
display(brand_scale,uneven_brand_scale)

Scale(('#FFAA15', 0.0), ('#15FFAA', 0.3333333333333333), ('#15AAFF', 0.6666666666666666), ('#FF15AA', 1))

Scale(('#FFAA15', 0), ('#15FFAA', 0.75), ('#15AAFF', 0.9), ('#FF15AA', 1))

### Maps
Color Maps are useful for explicitly assigning variables to colors. Effectively this a dictionary of colors.

**NOTE**: This is the only mutable color object

In [26]:
cmap = cc.Map(
    {color.name:color for color in brand_colors},
    name = 'BrandColorMap'
)


print(cmap)
cmap

Map{'mustard': '#FFAA15', 'lime': '#15FFAA', 'sky': '#15AAFF', 'pink': '#FF15AA'}


mustard,
lime,
sky,
pink,


### Saving and loading color objects
Colors and Color objects can be saved as JSON files which are easily shared between applications and other frameworks.

In [27]:
# Save several colors / color objects to the current directory
with TemporaryDirectory() as tempdir:
    pink.dump_json(f'{tempdir}/pink.json', overwrite=True)
    cmap.dump_json(f'{tempdir}/brand_map.json', overwrite=True)

    # Loading objects back into memory

    pink_reloaded = cc.Hex.load_json(f'{tempdir}/pink.json')
    display(pink_reloaded)

    # The default color type can be changed at load or globally
    ## At load
    cmap_reloaded = cc.Map.load_json(f'{tempdir}/brand_map.json', color_type='HSL')
    print(cmap_reloaded)
    
    ## Change global settings
    cc.settings.default_color_type = 'RGB'
    cmap_reloaded = cc.Map.load_json(f'{tempdir}/brand_map.json')
    print(cmap_reloaded)


'#FF15AA'

Map{'mustard': (38.205128, 1.0, 0.541176), 'lime': (158.205128, 1.0, 0.541176), 'sky': (201.794872, 1.0, 0.541176), 'pink': (321.794872, 1.0, 0.541176)}
Map{'mustard': (255, 170, 21), 'lime': (21, 255, 170), 'sky': (21, 170, 255), 'pink': (255, 21, 170)}


## Camps
Color camps are entire collections of colors and meant for larger organizational themes. The entire Camp can be saved and reloaded at once to quickly share colors and color objects between scripts, applications, and frameworks. All color objects in a camp must have a name attribute assigned.

The individual colors and color objects are saved in the following structure:

```
camp_name
|---colors
|   |--- ...
|---maps
|   |--- ...
|---palettes
|   |--- ...
|---scales
    |--- ...

```

In [28]:
project_camp = cc.Camp(
    name = 'ExampleCamp',
    description = 'This is how we can organize colors to represent science more clearly!',
)

project_camp.colors.add(pink)
# Note: the `name` of this color is "super_pink"
project_camp.colors.add(new_pink) 
project_camp.colors.add(mustard)

project_camp.colors.super_pink

(255, 21, 170)

In [29]:
# Or you can add them in bulk
project_camp.add_objects([lime, sky, cmap, brand_colors, brand_scale])

display(project_camp.maps.BrandColorMap)
# They can also be accessed via the 'get item' syntax
project_camp.colors['pink']

mustard,
lime,
sky,
pink,


'#FF15AA'

Camps can be saved to be easily shared

In [30]:
with TemporaryDirectory() as tempdir:
    project_camp.save(directory=tempdir)
    # reload camp
    reloaded_project_camp = cc.Camp.load(name = 'ExampleCamp', directory=tempdir)

# Practical Example

In [31]:
# pip install pandas==2.2.0
# pip install plotly==5.9.0

import pandas as pd
import plotly.express as px

mtcars = pd.read_csv('https://gist.githubusercontent.com/ZeccaLehn/4e06d2575eb9589dbe8c365d61cb056c/raw/64f1660f38ef523b2a1a13be77b002b98665cdfe/mtcars.csv')
# Edit element of column header
mtcars.rename(columns={'Unnamed: 0':'brand'}, inplace=True)
mtcars.head()

,brand,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
0,Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
1,Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
3,Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
4,Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2


In [32]:
car_camp = cc.Camp(name='car_camp')

car_camp.maps.add(
    cc.Map(
        {
            4:cc.Hex('#0173b2'),
            6:cc.Hex('#de8f05'),
            8:cc.Hex('#029e73'),
        },
        name = 'Cylinders'
    )
)

In [ ]:
fig = px.box(
    mtcars, 
    x = 'cyl', 
    y = 'mpg',
    color = 'cyl', 
    color_discrete_map=car_camp.maps.Cylinders
)
display(fig)

fig = px.box(
    mtcars, 
    x = 'cyl', 
    y = 'wt',
    color = 'cyl', 
    color_discrete_map=car_camp.maps.Cylinders
)
display(fig)